In [2]:
import pandas as pd
import os
import re
import torch
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split



### PRE-PROCESSING

Per ottenere un dataset che possa essere utilizzato come training del nostro language model partiamo dai file conllu che contengono i testi annotati di wikipedia italiana. Da questi file vogliamo ottenere una struttura dati che per ogni frase riporti id, testo e indice di Gulpease (per ora).

In [3]:
#si ottengono i path di ogni file per il pretraining e si salvano in una lista

ds_directory = "data"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)
    
print(ds_files)

['data\\prova.conllu', 'data\\prova2.conllu', 'data\\text_all.txt']


In [4]:

def get_id(file_path, n_file):
    id_list = []
    current_id = ""
    for line in open(file_path, 'r',  encoding='utf-8'):
        if line.startswith("# sent_id"):
            current_id = re.sub(r'\D', '', line)
            id_list.append(current_id + "_" + str(n_file))
    return id_list


In [5]:


#PROBLEMA: l'id non è univoco se abbiamo un insieme di file che sono stati "parsati" singolarmente. 
#Possibile soluzione: aggiungere un numero identificativo del file. ?! 

In [6]:
#le preposizioni articolate vanno divise in costituenti o va bene la frase intera? 
# -> intanto prendo la frase intera in caso cambio
def get_text(file_path):
    text_list = []
    current_sent = ""
    for line in open(file_path, 'r',  encoding='utf-8'): 
        if line.startswith("# text"):
            current_sent = line[9:].rstrip('\n')
            text_list.append(current_sent)
    return text_list
        

In [7]:
def comp_gulpease(ns, nw, nl):
    g_value = 89 + ((300*ns - 10*nl)/nw) #è corretta questa formula?
    return g_value

def get_gulpease(file_path):
    gulp_list = []
    for line in open(file_path, 'r', encoding = "utf-8"):  #questa riga la ripeto tre volte meglio ottimizzare in un'unica f
        if line.startswith("# text"):
            current_sent = line[9:].rstrip('\n')
            words = current_sent.split()
            gulp = comp_gulpease(1, len(words), sum(len(word) for word in words))
            gulp_list.append(gulp)
    return gulp_list

#DOMANDA: l'indice di gulpease serve a determinare la difficoltà di un intero testo. Come fare se abbiamo una singola frase?


In [8]:
#si estraggono id e testo tramite le funzioni sopra definite

id_list = []
gulpease_list = []
text_list = []
n_file = 1
for item in ds_files:
    id_list = id_list + get_id(item, n_file)
    n_file += 1
    text_list = text_list + get_text(item)
    gulpease_list = gulpease_list + get_gulpease(item)

In [9]:
#si crea un dataframe con una riga per frase, attributi: id, testo e indice di gulpease

ds_df = pd.DataFrame(columns=["id", "text", "gulp_index"])


In [10]:
ds_df["id"] = id_list
ds_df["text"] = text_list
ds_df["gulp_index"]  = gulpease_list

In [11]:
ds_df.head()

,id,text,gulp_index
0,1_1,L'allunaggio è la discesa di un veicolo sulla ...,74.555556
1,2_1,"Si distingue tra allunaggio duro, cioè un impa...",42.703704
2,3_1,"Il programma Luna, partito nel 1959 con la son...",56.142857
3,4_1,"Luna 9, il 3 febbraio 1966, eseguì il primo at...",63.615385
4,5_1,"Il primo allunaggio di un essere umano, il 20 ...",45.000000


In [12]:
 # TO  DO 
#ottimizzare il codice: il for line in open... farlo in una funzione unica

In [13]:
#Divido in training e test set
dataset = Dataset.from_pandas(ds_df)


### TOKENIZATION

In questa sezione si importa il tokenizzatore col quale si tokenizza ciascuna frase nel formato necessario per Bert.

In [14]:
import tokenizers
import transformers
from transformers import BertTokenizer

In [15]:
#Per il tokenizer ho bisogno del testo in formato txt
text = '\n'.join(ds_df['text'])

with open('data/text_all.txt', 'w', encoding="utf-8") as file:
    file.write(text)

In [16]:
#Dobbiamo fare il pre-training del tokenizer???????????
'''
tokenizer = tokenizers.BertWordPieceTokenizer()

files = ["data/text_all.txt"]
         
tokenizer.train(files=files, vocab_size=20000)#argomenti vanno bene? BOH
tokenizer.enable_truncation(max_length=256)'''

'\ntokenizer = tokenizers.BertWordPieceTokenizer()\n\nfiles = ["data/text_all.txt"]\n         \ntokenizer.train(files=files, vocab_size=20000)#argomenti vanno bene? BOH\ntokenizer.enable_truncation(max_length=256)'

In [17]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")


In [18]:
#train_df

NameError: name 'train_df' is not defined

In [ ]:
#facciamo l'encoding di tutto il dataset tokenizzando frase per frase
def encode(sample):
    return tokenizer(sample["text"], padding=True, truncation=True, max_length=256, return_special_tokens_mask=True)

train_set = train_df["text"].map(encode, batched=True)
test_set = test_df["text"].map(encode, batched=True)

train_set.set_format('torch', columns=["input_ids", "attention_mask", "label"])
test_set.set_format('torch', columns=["input_ids", "attention_mask", "label"])


AttributeError: 'list' object has no attribute 'map'

In [ ]:
# Funzione che applica la tokenizzazione ai dati di training, dev e test
def tokenize(batch):
    tokens = tokenizer(batch['sentence'], padding=True, truncation=True, max_length=512)
    tokens['label'] = [float(label) for label in batch["label"]]
    return tokens

train = train.map(tokenize, batched=True)
dev = dev.map(tokenize, batched=True)
test = test.map(tokenize, batched=True)

train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
dev.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_set[0]

{'input_ids': [102, 178, 1553, 12932, 12332, 160, 198, 146, 14577, 120, 141, 6964, 602, 11456, 697, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'special_tokens_mask': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

### TRAINING DI BERT

Si procede al training di Bert. Il modello dovrà partire da uno stato iniziale con pesi random, per questo non si importa il modello già addestrato, ma si configura semplicemente l'architettura la sua architettura per poi addestrarlo da zero. Si definisce poi una strategia di training e i suoi argomenti per poi addestrare il modello sul trask di Language Modeling. 

In [ ]:
from transformers import Trainer, TrainingArguments, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

In [ ]:
#utilizzeremo un bert piccolino per ora: prajjwal1/bert-mini

In [ ]:
model_name = "prajjwal1/bert-mini"
model_config = BertConfig.from_pretrained(model_name)

print(model_config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
model = BertForMaskedLM(model_config)

In [ ]:
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
#usiamo il datacollator per fare le batch per il training
datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.2, return_tensors="pt")

In [ ]:
#argomenti provvisori, da definire meglio
training_args = TrainingArguments(
    output_dir = "my_pretrained_model",
    evaluation_strategy="steps",
    logging_steps=10,
    overwrite_output_dir=True,
    num_train_epochs=2,
    save_steps=10,
    save_total_limit=2)

In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=10,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_names=None,
label_smoothing_facto

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=datacollator,
    train_dataset=train_set,
    eval_dataset=test_set)

In [ ]:
trainer.train()

c:\Users\bergo\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 172
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 44


  0%|          | 0/44 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.